In [4]:
import pandas as pd
import numpy as np
import re
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
from nltk.corpus import wordnet as wn

In [5]:
data_tweets = pd.read_csv(r'dataset_twitter.csv')

In [6]:
data_tweets.head()

,full_text,created_at,tweet_id,user_id,user_screen_name,user_location,user_followers,user_friends,user_favourites,user_statuses,label
0,@pasandoavisitar +eventos así. Cuál exclusión ...,11/01/2022 19:13,1481056605890351107,221554338,TeamMayitas,México,409,429,21883,10623,1
1,Está pasando que hay miedo entre los inoculado...,11/01/2022 19:10,1481055968062590982,848985083302350849,lonocomun,NaN,5,10,22,606,0
2,Desarrollador de videojuegos muere de covid de...,11/01/2022 19:10,1481055964069773312,164693617,ATVLatino,"Texas, USA",64610,20336,3328,83305,0
3,Creo que me voy a morir más de estrés y ansied...,11/01/2022 19:10,1481055846251614220,104063949,albertgaunt,Mexico City,572,557,13839,106862,1
4,Mi mayor miedo del COVID es perder el GUSTO y ...,11/01/2022 19:09,1481055687170215937,948665427898634242,DCEliane99,Insta: dai2204,94,157,21741,8607,0


## 1. Eliminar caracteres Especiales
En esta etapa se define la funcion clean_tweets() la cual elimina los carateres especiales, emojis, hashtags, urls y menciones de otros usuarios. Se convierte el texto a minúsculas. También se reemplazan algunos terminos usualmente usados en las redes sociales con el significado.

In [7]:
url = ('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|'
        '[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')
menciones = '@[\w\-]+'
hashtag = '#[\w\-]+'
caracteres_especiales = r'\W'
varios_espacios= r'\s+'

def clean_tweet(tweet):
    tweet_procesado = tweet.lower() 
    tweet_procesado = re.sub(menciones, ' ', tweet_procesado)
    tweet_procesado = re.sub(url, ' ', tweet_procesado)
    tweet_procesado = re.sub(caracteres_especiales, ' ', tweet_procesado)
    tweet_procesado = re.sub(" q ", ' que ', tweet_procesado)
    tweet_procesado = re.sub(" pq ", ' porque ', tweet_procesado)
    tweet_procesado = re.sub(" xq ", ' porque ', tweet_procesado)
    tweet_procesado = re.sub(varios_espacios, ' ', tweet_procesado, flags=re.I)
    return tweet_procesado

Limpiando nuestro dataset

In [8]:
for index, row in data_tweets.iterrows():
    cleaned = clean_tweet(row['full_text'])
    data_tweets.at[index, 'full_text'] = cleaned

In [9]:
data_tweets['full_text']

0        eventos así cuál exclusión por maternar antes...
1       está pasando que hay miedo entre los inoculado...
2       desarrollador de videojuegos muere de covid de...
3       creo que me voy a morir más de estrés y ansied...
4       mi mayor miedo del covid es perder el gusto y ...
                              ...                        
1324    lo que más odio de la traición de mi ex es la ...
1325    el otro día vi un post que decía decirle a alg...
1326          me ha dado todo covid depresión y ansiedad 
1327    me siento mal y no sé si son los síntomas de l...
1328    bajé 3 kilos esta semana gracias covid y depre...
Name: full_text, Length: 1329, dtype: object

## 2. Tokenizacion y eliminación de palabras vacías

Las palabras vacías o stopwords son palabras no tienen un significado por sí solas, sino que modifican o acompañan a otras, este grupo suele estar conformado por artículos, pronombres, preposiciones, adverbios e incluso algunos verbos.

In [10]:
print(stopwords.words("spanish"))

['de', 'la', 'que', 'el', 'en', 'y', 'a', 'los', 'del', 'se', 'las', 'por', 'un', 'para', 'con', 'no', 'una', 'su', 'al', 'lo', 'como', 'más', 'pero', 'sus', 'le', 'ya', 'o', 'este', 'sí', 'porque', 'esta', 'entre', 'cuando', 'muy', 'sin', 'sobre', 'también', 'me', 'hasta', 'hay', 'donde', 'quien', 'desde', 'todo', 'nos', 'durante', 'todos', 'uno', 'les', 'ni', 'contra', 'otros', 'ese', 'eso', 'ante', 'ellos', 'e', 'esto', 'mí', 'antes', 'algunos', 'qué', 'unos', 'yo', 'otro', 'otras', 'otra', 'él', 'tanto', 'esa', 'estos', 'mucho', 'quienes', 'nada', 'muchos', 'cual', 'poco', 'ella', 'estar', 'estas', 'algunas', 'algo', 'nosotros', 'mi', 'mis', 'tú', 'te', 'ti', 'tu', 'tus', 'ellas', 'nosotras', 'vosotros', 'vosotras', 'os', 'mío', 'mía', 'míos', 'mías', 'tuyo', 'tuya', 'tuyos', 'tuyas', 'suyo', 'suya', 'suyos', 'suyas', 'nuestro', 'nuestra', 'nuestros', 'nuestras', 'vuestro', 'vuestra', 'vuestros', 'vuestras', 'esos', 'esas', 'estoy', 'estás', 'está', 'estamos', 'estáis', 'están', 'e

In [11]:
def procesar_texto(cadena):
    palabras = cadena.split() 
    cadena_limpia = [palabra for palabra in palabras if palabra.lower() not in stopwords.words("spanish")]
    return cadena_limpia

In [12]:
for index, row in data_tweets.iterrows():
    cleaned = procesar_texto(row['full_text'])
    cleaned = ' '.join(cleaned)
    data_tweets.at[index, 'full_text'] = cleaned.strip()

In [13]:
data_tweets['full_text']

0       eventos así cuál exclusión maternar pandemia f...
1       pasando miedo inoculados miserables siguen aca...
2       desarrollador videojuegos muere covid después ...
3                    creo voy morir estrés ansiedad covid
4       mayor miedo covid perder gusto olfato gusto ha...
                              ...                        
1324    odio traición ex manera destrozó autoestima se...
1325    día vi post decía decirle alguien depresión tr...
1326                        dado covid depresión ansiedad
1327    siento mal sé si síntomas depresión corazón ro...
1328          bajé 3 kilos semana gracias covid depresión
Name: full_text, Length: 1329, dtype: object

## 3. Lemmatización

In [14]:
import spacy
import spacy_spanish_lemmatizer

In [15]:
nlp = spacy.load("es_core_news_sm")
nlp.replace_pipe("lemmatizer", "spanish_lemmatizer")

In [60]:
for token in nlp('He estado llorando todo el día'):
    print(f'palabra: {token.text} \t lemma:{token.lemma_}')

word: He 	lemma: haber
word: estado 	lemma: estar
word: llorando 	lemma: llorar
word: todo 	lemma: todo
word: el 	lemma: el
word: día 	lemma: día


In [16]:
for index, row in data_tweets.iterrows():
    texto_lemma = nlp(row['full_text'])
    lemmas = []
    for token in nlp(texto_lemma):
        lemmas.append(token.lemma_)
    data_tweets.at[index, 'full_text'] = ' '.join(lemmas)

In [17]:
data_tweets['full_text']

0       evento así cuál exclusión maternar pandemia fo...
1       pasar miedo inoculado miserable seguir acatar ...
2       desarrollador videojuegos morir covid después ...
3                   creer voy morir estrés ansiedad covid
4       mayor miedo covid perder gusto olfato gusto ha...
                              ...                        
1324    odio traición ex manera destrozar autoestima s...
1325    día ver post decir decirle alguien depresión t...
1326                        dado covid depresión ansiedad
1327    sentar mal saber si síntomas depresión corazón...
1328           bajar 3 kilo semana gracia covid depresión
Name: full_text, Length: 1329, dtype: object

In [18]:
data_tweets.to_csv(r'dataset_twitter_cleaned.csv', index = False, header=True)